In [1]:
import os
import json
import pandas as pd

os.listdir("./results")

# ['single_view_fp_plain_-8054598309511539763.json',
#  'single_view_emb_variational_5282197061669233432.json',
#  'multi_view_variational_4453866053670671622.json',
# ...

# e.g.
{
    "experiment": "multi_view_plain_-46151256717262646",
    "learning_rate": 0.0005,
    "batch_size": 32,
    "depth": 1,
    "hidden_dim": 256,
    "latent_dim": 256,
    "dropout_prob": 0.3,
    "best_valid_loss": 0.3437898400660655,
    "test_loss": 0.3843511521654285,
    "num_trainable_params": 4510870,
    "runtime": 201.72464776039124
}

# Iterate over all json files in the results directory and create dataframe
results = []
for file in os.listdir("./results"):
    with open(f"./results/{file}") as f:
        results.append(json.load(f))

df = pd.DataFrame(results)
df.head(20)

,experiment,learning_rate,batch_size,depth,hidden_dim,latent_dim,dropout_prob,best_valid_loss,test_loss,num_trainable_params,runtime,kl_weight
0,single_view_fp_plain_-8054598309511539763,0.0001,64,3,128,1024,0.1,0.534829,0.600784,1173396,60.185689,NaN
1,single_view_emb_variational_5282197061669233432,0.0010,128,1,256,256,0.1,0.937528,1.099271,792576,80.204371,0.010
2,multi_view_variational_4453866053670671622,0.0005,32,1,256,256,0.3,0.832869,0.867915,5037206,213.371477,0.010
3,single_view_emb_variational_-7401032702674798705,0.0005,32,3,128,256,0.3,1.622857,1.662488,432640,83.216578,0.100
4,single_view_fp_variational_-1614181474134004323,0.0005,32,2,128,256,0.3,0.445021,0.478593,1007764,106.491475,0.001
5,single_view_fp_plain_-1363216830430859170,0.0005,32,2,512,256,0.1,0.384676,0.395282,4514452,102.039744,NaN
6,single_view_fp_variational_6164373553531490582,0.0005,128,3,128,256,0.1,5.535143,5.722931,1041300,71.915485,0.100
7,multi_view_variational_7392475774628741531,0.0005,64,3,128,512,0.3,0.437350,0.487291,2670998,125.670840,0.001
8,single_view_emb_plain_-8030435951507706559,0.0001,64,3,128,256,0.3,0.325011,0.460477,366592,105.231596,NaN
9,single_view_emb_plain_8398463583242479563,0.0010,128,1,256,256,0.1,0.364252,0.521808,660992,48.325668,NaN


In [3]:
# sort by best validation loss
df.sort_values("best_valid_loss").head(20)

,experiment,learning_rate,batch_size,depth,hidden_dim,latent_dim,dropout_prob,best_valid_loss,test_loss,num_trainable_params,runtime,kl_weight
4403,multi_view_plain_6889676395874673667,0.0010,128,3,128,512,0.3,0.279012,0.388109,2538902,226.549141,NaN
1854,multi_view_plain_-6980364999454184749,0.0005,64,2,128,1024,0.3,0.283760,0.380885,2934166,259.208587,NaN
2709,multi_view_plain_-4039478511947688627,0.0005,128,2,128,512,0.3,0.286358,0.400966,2404758,211.198660,NaN
3927,multi_view_plain_-1152802831377065867,0.0010,128,2,128,1024,0.1,0.290226,0.394491,2934166,71.499071,NaN
2084,multi_view_plain_121099320718851654,0.0005,64,2,128,256,0.1,0.293108,0.389307,2140054,213.675427,NaN
5008,multi_view_plain_5510231668160124536,0.0005,128,1,256,256,0.3,0.294341,0.420666,4510870,249.576968,NaN
1838,multi_view_plain_-4067751348410180787,0.0005,64,3,128,1024,0.3,0.295221,0.396528,3068310,180.527764,NaN
1469,multi_view_plain_-2749008768293101802,0.0010,64,2,128,256,0.1,0.295709,0.367693,2140054,177.851590,NaN
1560,single_view_emb_plain_8658592234400222681,0.0001,64,2,128,512,0.3,0.298029,0.482521,399104,104.644583,NaN
201,multi_view_plain_-4446959247277792444,0.0005,64,2,128,512,0.3,0.299130,0.366866,2404758,170.107697,NaN


In [17]:
subdf = df[df["experiment"].str.contains("multi_view_variational")].sort_values("best_valid_loss").iloc[:100]

for item in ["learning_rate", "batch_size", "depth", "hidden_dim", "latent_dim", "dropout_prob", "kl_weight"]:
    print(subdf[item].value_counts())

# give avg runtime
print(subdf["runtime"].mean())

learning_rate
0.0001    52
0.0005    35
0.0010    13
Name: count, dtype: int64
batch_size
32     71
64     24
128     5
Name: count, dtype: int64
depth
1    38
2    33
3    29
Name: count, dtype: int64
hidden_dim
128    65
256    24
512    11
Name: count, dtype: int64
latent_dim
256     42
512     32
1024    26
Name: count, dtype: int64
dropout_prob
0.1    57
0.3    43
Name: count, dtype: int64
kl_weight
0.001    96
0.100     4
Name: count, dtype: int64
330.3722142481804


In [9]:
MSE = 0.388109
root_MSE = MSE ** 0.5
root_MSE

0.62298394842885

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# adj batch_size x num_nodes x num_nodes
num_nodes = 60
batch_size = 32
hidden_dim = 256
x = torch.randn(batch_size, hidden_dim)
adj = torch.randn(batch_size, num_nodes, num_nodes)
print(adj.shape)
print(x.shape)

to_adj = nn.Linear(hidden_dim, 2*num_nodes*(num_nodes-1)//2)

x = to_adj(x)
print(x.shape)
x = torch.reshape(x, (x.size(0), -1, 2))
print(x.shape)
x = F.gumbel_softmax(x, tau=1, hard=True)[:,:,0]
print(x.shape)

adj = torch.zeros(x.size(0), num_nodes, num_nodes)   # initialize adjacency matrix
idx = torch.triu_indices(num_nodes, num_nodes, 1)                     # get upper triangular indices
adj[:,idx[0],idx[1]] = x                              # fill in upper triangular part
adj = adj + torch.transpose(adj, 1, 2)                                      # make symmetric
adj.shape # (batch_size, n_nodes, n_nodes)

torch.Size([32, 60, 60])
torch.Size([32, 256])
torch.Size([32, 3540])
torch.Size([32, 1770, 2])
torch.Size([32, 1770])


torch.Size([32, 60, 60])

tensor([[ 0,  0,  0,  ..., 57, 57, 58],
        [ 1,  2,  3,  ..., 58, 59, 59]])

In [19]:
x

tensor([[0., 1., 0.,  ..., 1., 0., 0.],
        [1., 1., 0.,  ..., 1., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 1.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 1., 1.,  ..., 1., 1., 1.]], grad_fn=<SelectBackward0>)

In [24]:
60*59//2

1770

In [26]:
num_nodes*(num_nodes-1)//2

1770

In [20]:
60*60

3600

In [10]:
adj.view(x.size(0), -1).shape

torch.Size([32, 3600])

In [6]:
adj

tensor([[[ 0.1364, -0.3604, -1.8680,  ..., -0.4514,  1.6060,  1.2494],
         [ 0.3040, -3.0094,  1.1780,  ...,  0.7238,  0.9798,  2.0547],
         [-1.0952,  1.3516,  1.1619,  ..., -1.5530, -0.6029,  0.6771],
         ...,
         [-0.3432,  0.0144, -2.3176,  ..., -1.4380, -0.8211,  0.5559],
         [ 1.7502,  1.9408, -0.3727,  ...,  1.1111, -0.3406,  0.9098],
         [-0.9984, -0.4357, -2.2276,  ...,  1.3976, -0.0488, -1.7852]],

        [[-1.8168, -0.5372,  1.7354,  ..., -0.0310, -0.8994, -0.6736],
         [ 0.5633, -0.3980, -0.2511,  ..., -1.6547,  1.0410,  0.0226],
         [ 0.7749, -1.4277,  0.2565,  ...,  1.4209, -0.0270, -0.2470],
         ...,
         [ 0.7217,  0.0438, -1.5403,  ..., -0.7871, -0.3613,  0.3442],
         [ 0.9818, -0.5376,  0.5828,  ..., -0.5802, -0.7084,  0.6669],
         [-0.4688,  1.3937,  1.0418,  ..., -0.0287,  0.2791, -0.5498]],

        [[ 0.8040, -0.1471, -0.4128,  ...,  0.7954, -0.5344, -0.3658],
         [-1.2603, -1.0739, -1.5132,  ..., -0